In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision

import os
import argparse

from models.densenet import *

In [2]:
from utils import *

In [3]:
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import random

----------------------------

In [5]:
device = torch.device('cuda:3') if torch.cuda.is_available() else 'cpu'
best_loss = 100  # best test accuracy
start_epoch = 0

In [6]:
exp_dir = os.path.join("./experiments/dense_fs", f"lr_{1e-3}")

if not os.path.isdir(exp_dir):
    os.makedirs(exp_dir)

---------------------------------------------------
### Data Preparing

In [7]:
print('==> Preparing data..')

==> Preparing data..


In [8]:
feature_path = './data/LLC_FS_latents.h5'
label_path = './data/LLC_FS_r0.5A.parquet'

In [9]:
llcfs_dataset_train = LLCFSDataset(feature_path, label_path, 'train')
llcfs_dataset_valid = LLCFSDataset(feature_path, label_path, 'valid')

In [10]:
para_norm = llcfs_dataset_train.para_norm

In [11]:
batch_size = 256

In [12]:
train_loader = llcfs_loader(feature_path, label_path, batch_size, 'train')
test_loader = llcfs_loader(feature_path, label_path, batch_size, 'test')

--------------------------------------------
### Model

In [13]:
print('==> Building model..')

==> Building model..


In [14]:
# Model
net = densenet_fs()
net = net.to(device)

In [15]:
if device == torch.device('cuda:3'):
    net = torch.nn.DataParallel(net, device_ids=[3])
    cudnn.benchmark = True

In [16]:
criterion = nn.L1Loss()

In [17]:
optimizer = optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-4)

In [18]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2)

------------------------------------------------
### Train and Evaluation

In [19]:
def train(epoch, visual_use=False):
    print('\nTrain Epoch: %d' % epoch)
    net.train()
    train_loss, steps_train = 0, 0
    for inputs, targets in tqdm(train_loader):
        steps_train += 1
        inputs = inputs.view(-1, 1, 16, 16).contiguous()
        targets = targets.view(-1, 1).contiguous()
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        #template_print = 'Step(train): {:d} Loss: {:.3f}'
        #print(template_print.format(steps_train, train_loss / steps_train))
    if visual_use:
        train_dict = {'train_loss': train_loss / steps_train}
        experiment.log_metrics(train_dict, epoch=epoch)
    template_print = 'Epoch(train): {:d} Loss: {:.3f}'
    print(template_print.format(epoch, train_loss / steps_train))

In [20]:
def test(epoch, visual_use=False):
    global best_loss
    print('\nTest Epoch: %d' % epoch)
    net.eval()
    test_loss, steps_test = 0, 0
    with torch.no_grad():
        for inputs, targets in tqdm(test_loader):
            steps_test += 1
            inputs = inputs.view(-1, 1, 16, 16).contiguous()
            targets = targets.view(-1, 1).contiguous()
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
        if visual_use:
            test_dict = {'test_loss': test_loss / steps_train}
            experiment.log_metrics(test_dict, epoch=epoch)
    template_print = 'Epoch(test): {:d} Loss: {:.3f}'
    print(template_print.format(epoch, test_loss / steps_test))
    
    # Save checkpoint
    if test_loss < best_loss:
        print('Saving...')
        state = {
            'net': net.state_dict(),
            'loss': test_loss, 
            'epoch': epoch
        }
        check_dir = os.path.join(exp_dir, 'ckpt.pth')
        torch.save(state, check_dir)
        best_loss = test_loss

In [21]:
def prediction(feature_sample, para_norm):
    print('\nPrediction...')
    net.eval()
    with torch.no_grad():
        feature_sample_reshaped = feature_sample.view([-1, 1, 16, 16]).contiguous()
        feature_sample_reshaped.to(device)
        label_norm = net(feature_sample_reshaped)
    label_norm.cpu()
    mean_norm, std_norm = para_norm
    label_pred = label_norm * std_norm + mean_norm
    return label_pred

---------------------------
### Main

In [ ]:
if __name__ == '__main__':
    seed = 314
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    start_epoch = 1
    num_epochs = 3
    for epoch in range(start_epoch, start_epoch+num_epochs):
        train(epoch)
        test(epoch)
        scheduler.step()

------------------------------------
#### Prediction

In [23]:
samples_test = next(iter(test_loader))

In [58]:
label_pred = prediction(samples_test[0][10], para_norm)


Prediction...


In [59]:
label_pred

tensor([[1.2624e-13]], device='cuda:3')

In [61]:
label_true = samples_test[1][0] * para_norm[1] + para_norm[0]

In [63]:
print(f'label_pred: {label_pred[0][0]}, label_true: {label_true}.')

label_pred: 1.2624390465301727e-13, label_true: 1.5790011442459728e-13.
